In [1]:
import sys
sys.path.append('libmitigation')

In [2]:
#Preparing qiskit environment
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit import IBMQ
from qiskit import Aer
from scipy import optimize
import sys, time
import mpmath as mp
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
import qiskit.ignis.mitigation as mit
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter
import importlib
from nation_etal import NationEtal
from lib_grover import *

In [4]:
import nation_etal
importlib.reload(nation_etal)
from nation_etal import NationEtal

In [5]:
nbit = 5
noise_model = NoiseModel()
for qi in range(nbit+1):
    if qi >= 0:
        read_err = ReadoutError([[0.90, 0.10],[0.10,0.90]])
        noise_model.add_readout_error(read_err, [qi])

In [6]:
shots = 8192
qr = QuantumRegister(nbit+1)
mit_pattern = [[i] for i in range(nbit+1)]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)

In [7]:
meas_filter = NationEtal(nbit+1, meas_fitter.cal_matrices, mit_pattern)

In [8]:
cal_matrices = meas_filter.cal_matrices
cal_matrices

[array([[0.89831543, 0.09960938],
        [0.10168457, 0.90039062]]),
 array([[0.89672852, 0.09753418],
        [0.10327148, 0.90246582]]),
 array([[0.89501953, 0.10546875],
        [0.10498047, 0.89453125]]),
 array([[0.90356445, 0.09631348],
        [0.09643555, 0.90368652]]),
 array([[0.89379883, 0.09338379],
        [0.10620117, 0.90661621]]),
 array([[0.89916992, 0.09960938],
        [0.10083008, 0.90039062]])]

In [9]:
x_tilde = meas_filter.apply({"000000": 50, "111111": 50}, d=2)

{'000000': 0.5, '111111': 0.5}
Method by Nation, Kang, Sundaresan, and Gambatta
{'111111', '000000'}
main process: Done!
sum of mitigated probability vector x: 1.0000005350167278
[ 1.0137055  -0.10705123 -0.11206877  0.00935263 -0.10186232  0.00845383
  0.00884556 -0.11236926  0.00945005  0.00988983  0.00894144  0.00841914
 -0.10943872  0.00914056  0.00956517  0.00864571  0.00921817  0.00966469
  0.00818648  0.0080688   0.00857688 -0.10398889 -0.10798236  0.00902139
  0.0094404   0.00853328  0.0090331   0.00953832  0.00802197  0.00790687
  0.00840463 -0.10196327  0.00922625  0.00878716  0.00865749  0.0092027
 -0.1107091   0.00768775  0.008172   -0.09982879  0.00805484 -0.09786597
 -0.10388176  0.99119837]
start sgs_algorithm
number of positive values:  32


In [12]:
sum(x_tilde.values())

100.0

In [ ]:
meas_filter.prepare_nation_A_tilde(meas_filter.extend_keys({"000000", "111111"}, 2))

In [ ]:
meas_filter.A_tilde.shape

In [ ]:
np.sum(meas_filter.A_tilde)

In [ ]:
meas_filter.prepare_nation_A_tilde(meas_filter.extend_keys({"000000", "111111"}, 2))

In [ ]:
meas_filter.A_tilde.shape

In [ ]:
from draw_heatmap import draw_heatmap

In [ ]:
draw_heatmap(meas_filter.A_tilde)

In [ ]:
from itertools import combinations
import copy

def change_bit_at_poses(key: str, poses: int) -> str:
    for pos in poses:
        key = key[:pos] + "1" + key[pos+1:] if key[pos] == "0" else key[:pos] + "0" + key[pos+1:]
    return key

def extend_keys(original_keys: set, max_dist: int) -> set:
    
    extended_key_set = copy.deepcopy(original_keys)
    
    for key in original_keys:
        n = len(key)
        for d in range(max_dist):
            combs = combinations(range(n), d + 1)
            for comb in combs:
                new_key = change_bit_at_poses(key, comb)
                extended_key_set.add( new_key )
    return extended_key_set

In [ ]:
7 * 6 * 5 / (1 * 2 * 3) + 7 * 6 / (1 * 2) + 7 / 1 + 1

In [ ]:
len(extend_keys(set(["0000000", "1101100"]), 3))

In [ ]:
extend_keys(set(["0000000", "1101100"]), 3)